# Gensim Text Summarization
* 使用gensim實現文本摘要，其中gensim的summarize是用textRank演算法實作
* Jieba及CKIP做不同斷詞方式，會讓摘要有所不同
    - 這裡採用jieba_tw。有用繁體中文專門調教過的jieba版本，但依然不夠精準
    - CKIP在繁體中文的斷詞更精準

# TextSum_Gensim
* 將各種預處理的方法打包進class
    1. cut_sent: 斷句
    2. jieba_tokenize: 使用jieba進行斷詞
    3. ckip_tokenize: 使用ckip進行斷詞
    4. clean: 清理gensim summarize後的文本
    5. summarize: 進行預處理後，調用gensim summarize，再clean文本

In [1]:
'''Text Summarization based on gensim summarize module'''

import re
import jieba_tw
from ckiptagger import data_utils, construct_dictionary, WS
from gensim.summarization import summarize


class TextSum_Gensim:
    def cut_sent(self, para):
        '''用正規表示法實現斷句
            @input para(str): 文本資料
            @return 被斷開的句子(list)
        '''
        para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
        para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)
        para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)
        para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
        para = para.rstrip()
        return para.split("\n")
    
    def jieba_tokenize(self, sents):
        '''使用優化過的jieba_tw進行斷詞
            @input sents(list): 斷句後的list
            @return 斷詞後的整篇文章(str)
        '''
        sents_tokenized = [] #暫存被斷好詞的各個句子
        for sent in sents:
            seg_list = jieba_tw.cut(sent, cut_all=False)
            _sent = ' '.join(seg_list).rstrip('\n') # 用空格隔開各個詞
            sents_tokenized.append(_sent)
        
        # 把各斷好詞的句子，組合成整個字串，並在尾句加上.，供gensim分辨句子段落
        content = '. '.join(sents_tokenized)
        return content
    
    def ckip_tokenize(self, sents):
        '''使用CKIP進行斷詞
            @input sents(list): 斷句後的list
            @return 斷詞後的整篇文章(str)
        '''
        ws = WS("./ckip_data") # laod ckip ws model
    
        sents_tokenized = []
        for sent in sents:
            token = ws([sent])[0]
            _sent = ' '.join(token)
            sents_tokenized.append(_sent)

        content = '. '.join(sents_tokenized)
        return content
    
    def clean(self, summarization_content):
        '''將格式處理乾淨
            @input summarization_content(str)
            @return 清理格式後的summarization_content
        '''
        summarization_content = summarization_content.replace('.', '') # 刪除句子分隔.
        summarization_content = summarization_content.replace(' ', '') # 刪除空格
        summarization_content = summarization_content.replace('\n', '') # 刪除換行符號
        return summarization_content
    
    def summarize(self, paragraph, tokenize_mode):
        '''將前處理及gensim summarize整合在一起
            @input paragraph(str): 原始文章資料
            @input tokenize_mode(str): 使用jieba或CKIP進行斷詞
            @return 文本摘要
        '''
        # preprocessing
        sents = self.cut_sent(paragraph) # 斷句
        
        # 斷詞
        token_content = ''
        if tokenize_mode == 'jieba':
            token_content = self.jieba_tokenize(sents)
        elif tokenize_mode == 'ckip':
            token_content = self.ckip_tokenize(sents)
            
        # gensim summarize
        sum_paragraph = summarize(token_content)
        
        # 清除格式
        sum_paragraph = self.clean(sum_paragraph)
        
        return sum_paragraph

//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1)

# Get Started
用以下這篇新聞作文文本，測試文本摘要
    - https://www.bbc.com/zhongwen/trad/world-51993469

In [2]:
text = (
    "為了遏制新冠病毒的傳播，意大利北部疫情最嚴重的倫巴底大區（Lombardy）採取了更嚴格的防控措施。"
    "據意大利星期六（3月21日）晚間宣佈的新規定，當局禁止居民在戶外做運動，禁止使用自動售貨機，露天市集亦關閉。"
    "意大利周六新增新冠肺炎確診病例6557例，新增死亡病例793例，均為疫情爆發以來的最高數值。"
    "意大利是歐洲疫情最嚴重的國家，也是世界上因新冠病毒死亡人數最多的國家。意大利累計確診病例增至53578例，累計死亡病例達4825例。"
    "倫巴底大區是該國受影響最嚴重的地區，有3095人死亡。"
    "倫巴底大區主席阿蒂裏奧·豐塔納在一份聲明中宣佈了有關新措施。聲明要求，除了「基本」供應鏈外，停止其他所有企業活動。"
    "除在醫院、公路和鐵路上工作的人員外，將停止所有建築工地的開工。此外，將暫停開放所有露天集市。"
    "自3月8日以來，倫巴第大區就一直處於封鎖狀態，當局希望在那裏首先看到抗疫結果。"
    "為了進一步遏制疫情傳播，意大利總理孔特當地時間周六晚發表電視講話，要求全國暫停所有「非必要的」非關鍵性生產活動。但他沒有具體說明哪些生產活動被視為必不可少。"
    "此外，藥店和超市將繼續營業，郵局和銀行等服務部門不受影響。公共交通將繼續運行。"
    "孔特在全國電視講話中說，意大利會將生產力速度放慢，但不會停止。孔特並稱，意大利正面臨該國自二戰以來最嚴重的危機。"
    "儘管意大利目前採取了嚴格的防控措施，但新確診病例和死亡病例仍在繼續增長。"
    "全球多國已有新冠肺炎確診病例逾22萬例，並有9000多人死亡。"
    "西班牙是目前歐洲地區除意大利外疫情最嚴重的國家。"
    "西班牙衛生部通報，當地時間周六，西班牙累計新冠肺炎確診病例達24926例，死亡病例達1326人，死亡人數激增32%。"
    "西班牙首相桑切斯在當地周六晚間舉行的新聞發佈會上警告說，西班牙疫情「最壞的日子還未到來」，「非常艱難的日子即將來臨」。"
    "西班牙政府此前已在全國範圍內實施「封城」措施，除工作、購買食品藥物、就醫、遛狗及其他必須和緊急情況外，所有居民和私人車輛禁止外出，共有4600萬人受到「封城」措施影響。"
    "此外，西班牙還關閉全國除超市、藥店等生活必需產品銷售和服務外的公共場所。"
    "世界多國領導人和政治人物敦促公民要遵守規則。"
    "英國首相鮑里斯·約翰遜發出警告，除非國民共同行動，採取行動減緩病毒的傳播，否則英國國民醫療保健服務將「不堪重負」。"
    "約翰遜並敦促民眾遵循社交疏離的建議，不要在星期日當地傳統的母親節探訪親屬，最好避免非必要的身體接觸，改為打電話或視像通話。"
    "截至周六，英國累計新冠肺炎確診病例超過5000例，超過230人死亡。"
    "英格蘭國民醫療保健計劃將向多達150萬在疫情中面臨風險的人士發信，強烈建議他們在未來12周呆在家中保護自己。"
    "這些面臨風險的人士包括接受器官移植者、嚴重呼吸道疾病患者或特定癌症患者。"
)

In [3]:
tg = TextSum_Gensim()

# 用jieba進行斷詞的摘要
summarization = tg.summarize(text, 'jieba')
print(summarization)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/f7/q6c0fqv12z53z21nptvyyr040000gn/T/jieba.cache
Loading model cost 0.606 seconds.
Prefix dict has been built succesfully.


為了遏制新冠病毒的傳播，意大利北部疫情最嚴重的倫巴底大區（Lombardy）採取了更嚴格的防控措施。為了進一步遏制疫情傳播，意大利總理孔特當地時間周六晚發表電視講話，要求全國暫停所有「非必要的」非關鍵性生產活動。西班牙是目前歐洲地區除意大利外疫情最嚴重的國家。西班牙衛生部通報，當地時間周六，西班牙累計新冠肺炎確診病例達24926例，死亡病例達1326人，死亡人數激增32%。


In [4]:
# 用ckip進行斷詞的摘要
summarization = tg.summarize(text, 'ckip')
print(summarization)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

為了遏制新冠病毒的傳播，意大利北部疫情最嚴重的倫巴底大區（Lombardy）採取了更嚴格的防控措施。意大利周六新增新冠肺炎確診病例6557例，新增死亡病例793例，均為疫情爆發以來的最高數值。西班牙是目前歐洲地區除意大利外疫情最嚴重的國家。西班牙衛生部通報，當地時間周六，西班牙累計新冠肺炎確診病例達24926例，死亡病例達1326人，死亡人數激增32%。
